In [99]:
import numpy as np
import urllib
import librosa
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from microphone import record_audio
from IPython.display import Audio
from pathlib import Path
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion 
from scipy.ndimage.morphology import iterate_structure
import pickle
from collections import Counter, defaultdict
# make this a glboal variable - it is false unless audio_to_digital changes
Checking=False 

In [113]:
def audio_to_digital(*file_path_or_music):
    if(len(file_path_or_music)!= 0):
        """if file_path_or_music is Path:
            local_song_path=file_path_or_music[0]
            frames, sample_rate = librosa.load(local_song_path, sr=44100, mono=True, duration=5) #Seconds
            audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])"""
        if(".mp3" in file_path_or_music[0]):
            #analyze file input if it is there
            from pathlib import Path
            local_song_path = Path(file_path_or_music[0])
            frames, sample_rate = librosa.load(local_song_path, sr=44100, mono=True, duration=5) #Seconds
            audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])

        else:
            #analyze file input if it is there
            with open(file_path_or_music[0], 'r') as R:
            # each sample is written to a single line in the text file
            # this reads them in as a single integer-valued numpy array
                audio_data = np.asarray([int(i) for i in R])
    else:
        #or else we are recording what they say
        from microphone import record_audio
        listen_time = 10  # seconds
        frames, sample_rate = record_audio(listen_time)
        # read in the recorded audio, saved as a numpy array of 16-bit integers
        audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])
    return audio_data

In [114]:
def digitalToSpecto(audio):
    S, freqs, times = mlab.specgram(audio, NFFT=4096, Fs=44100,
                                  window=mlab.window_hanning,
                                  noverlap=4096 // 2)
    return(S)

In [115]:
def spectrogram_to_peaks(arr):
    """ 
    Creates a boolean array showing peaks, given data from a spectrogram.
    
    Parameters:
        arr: The array produced by the spectrogram from digital_to_spectrogram with shape (N,M)
        
    Returns:
        peaks: A boolean array with shape (N,M). Peaks in the data are where peaks == True.
    """

    # Creating the histogram
    arr_flattened = np.log(arr.flatten() + 1e-20)
    
    N = arr_flattened.size # Number of elements in the array
    cnt, bin_edges = np.histogram(arr_flattened, bins=N//200, density=True)
    bin_width = np.diff(bin_edges) 
    
    ## print(np.sum(cnt*bin_width)) # check that summation = 1
    
    # Creating the cumulative distribution
    cumulative_distr = np.cumsum(cnt*bin_width)
    
    # Defining the cutoff
    frac_cut = 0.9
    bin_index_of_cutoff = np.searchsorted(cumulative_distr, frac_cut)
    
    # given the bin-index, we want the associated log-amplitude value for that bin
    cutoff_log_amplitude = bin_edges[bin_index_of_cutoff]
    
    # Defining the footprint
    fp = generate_binary_structure(rank=2,connectivity=2)
    fp = np.ones((4,5))
    
    peaks = ((arr > cutoff_log_amplitude) & (arr == maximum_filter(arr, footprint=fp)))
    
    return peaks

In [116]:
def peaks_to_dic_newsongs(local_peaks):
    """
    Creates a dictionary of fingerprints given the peaks from the previous function. Every time this function is called,
    it will initialize an empty dictionary and populate it with values from our pickled database
    to be compared against the fingerprint of the peaks that are currently being passed.
    
    Parameters:
        local_peaks: A (N,M) boolean array of peaks in the audio data
        
    Returns:
        None
    """
    
    # Compiling a new dictionary every time this function is called from the contents of the .pkl
    song_dic = {} 
    with open("songs.pkl", mode="rb") as opened_file:
        while True:
            try:
                entry = pickle.load(opened_file)
            except EOFError:
                break
            else:
                song_dic.update(entry)
    time,freq= np.where(local_peaks)
    song_name=input("Song Name= ")
    fanout = 15
    for i in range(freq.size-fanout):
        for j in range(1,fanout+1):
            finger=(freq[i], freq[i+j], time[i+j]-time[i])
            if finger in song_dic.keys():
                song_dir = (song_name, time[i])
                song_list_dic=song_dic[finger]
                song_list_dic.append(song_dir)
                song_dic.update([(finger, song_list_dic)])
         #       print("updated")
#                 previousfingers.append(song_dir) ## this is all extraneous previous code
#                 print((finger, previousfingers))
#                 song_dic.update([(finger, previousfingers)])
            else:
                song_dic[finger]=[(song_name, time[i])]      
    # open and save to a pkl file, mode = wb for binary storage w/ numpy array
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic,opened_file)
        # For every entry in the new dictionary,pickle the entry.
        # TODO: Figure out how to JUST append new items, rather than 
       # for key, value in song_dic.items():
           # print("it was put in")
            #pickle.append((key, value), opened_file)
    print("Done")

In [117]:
# test cell

song_dic={}
with open("songs.pkl", mode="rb") as opened_file:
    while True:
        try:
            entry = pickle.load(opened_file)
        except EOFError:
            break
        else:
            song_dic.update(entry)
k=0
for i in song_dic.values():
    k+=1
print(k)

142631


In [118]:
def peaks_to_dic_findsong(local_peaks):
    """
    Creates a dictionary of fingerprints given the peaks from the previous function. Every time this function is called,
    it will initialize an empty dictionary and populate it with values from our pickled database
    to be compared against the fingerprint of the peaks that are currently being passed.
    
    Parameters:
        local_peaks: A (N,M) boolean array of peaks in the audio data
        
    Returns:
        song: the most common song in the database which corresponds to the fingerprints of the audio sample
    """
    # Compiling a new dictionary every time this function is called from the contents of the .pkl
    song_dic2 = {}
    with open("songs.pkl", mode="rb") as opened_file:
        while True:
            try:
                entry = pickle.load(opened_file)
            except EOFError:
                break
            else:
                song_dic2.update(entry)
    time, freq= np.where(local_peaks)
    songList = []
    fingerList=[]
    abs_time=[]
    fanout=15
    for i in range(freq.size-fanout):
        for j in range(1,fanout+1):
            finger=(freq[i], freq[i+j], time[i+j]-time[i])
            abs_time.append(time[i])
            fingerList.append(finger)
    for i in range(len(fingerList)):
        if fingerList[i] in song_dic2:
            song_guess=song_dic2[fingerList[i]]
            # for every song associated with that fingerprint, find the absolute dt between that song's timebin
            # and the audio sample's time bin, and append both the name and dt to a list of songs
            for t in range(len(song_guess)):
                songList.append((song_guess[t][0], np.abs((song_guess[t][1]-abs_time[i]))))    
                #print (song_guess[t][0], (song_guess[t][1]-abs_time[i]))
    Counter_songs= Counter(songList)
    song =Counter_songs.most_common(3)
    return song

In [119]:
def final_function(*song):
    if(len(song)!=0):
        audio=audio_to_digital(song[0])
        arr=digitalToSpecto(audio)
        local_peaks=spectrogram_to_peaks(arr)
        peaks_to_dic_newsongs(local_peaks)
    else:
        audio=audio_to_digital()
        arr=digitalToSpecto(audio)
        local_peaks=spectrogram_to_peaks(arr)
        return peaks_to_dic_findsong(local_peaks)
    

In [126]:
final_function()

Using input device 'Microphone Array (Realtek High'
Recording ended


[(('Jungle', 38), 367),
 (('Hold Back The River', 8), 356),
 (('Jungle', 51), 355)]

In [108]:
#Testing and puting in the songs
from pathlib import Path
root=Path(".")
files=sorted(Path(root / "Songs").glob('*.mp3'))
for i in range(len(files)):
    print(files[i])
    final_function(files[i])

Songs\Alvaro Soler - Volar.mp3
Song Name= Volar
Done
Songs\berhana - Janet.mp3
Song Name= Janet
Done
Songs\Born To Be Wild.mp3
Song Name= Born To Be Wild
Done
Songs\Come with Me Now.mp3
Song Name= Come with Me Now
Done
Songs\Despacito.mp3
Song Name= Despacito
Done
Songs\Dont Stop me Now - Queen.mp3
Song Name= Dont Stop me Now
Done
Songs\DragonForce - Trough the Fire and Flames [HQ (Very High Audio Quality)].mp3
Song Name= Through the Fire and Flames
Done
Songs\Hold Back The River.mp3
Song Name= Hold Back The River
Done
Songs\HONNE - Day 1.mp3
Song Name= Honne
Done
Songs\Hyukoh - Gondry.mp3
Song Name= Gondry
Done
Songs\I Knew You Were Trouble..mp3
Song Name= I Knew You Were Trouble
Done
Songs\Imagine Dragons - Believer.mp3
Song Name= Believer
Done
Songs\Imagine Dragons, Khalid - Thunder Young Dumb & Broke (MedleyAudio).mp3
Song Name= Thunder Young Dumb & Broke
Done
Songs\Jungle.mp3
Song Name= Jungle
Done
Songs\My Life - Billy Joel.mp3
Song Name= My Life
Done
Songs\No Name - NF.mp3
Song 

In [107]:
with open("songs.pkl", mode="wb") as opened_file:
    pickle.dump({}, opened_file)

In [77]:
jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

SyntaxError: invalid syntax (<ipython-input-77-0950bf7915fd>, line 1)